# TF-IDF

This assessment introduces the TF-IDF algorithm for representing text as vectors.
We will implement TF, IDF and TF-IDF from scratch.

Goals:
- Understand the intuition behind TF-IDF
- Implement TF, IDF and TF-IDF manually
- Use TF-IDF for document similarity with cosine similarity


In [14]:
import re
import math
import numpy as np

## 1. Corpus

We will start with a small set of short documents.

**Task:** Define a corpus with at least 4 documents. Try to include repeated and rare words.


In [15]:
docs = [
    "Machine learning is powerful",
    "Deep learning is a branch of machine learning",
    "This document is about learning representations of text",
]

print("Number of documents:", len(docs))
for i, d in enumerate(docs, 1):
    print(f"{i}. {d}")


Number of documents: 3
1. Machine learning is powerful
2. Deep learning is a branch of machine learning
3. This document is about learning representations of text


## 2. Text preprocessing

We will:
- lowercase
- remove non alphabetic characters
- tokenize by whitespace

**Task:** Implement `tokenize(text)` returning a list of tokens.


In [16]:
def tokenize(text: str):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

tokenized_docs = [tokenize(d) for d in docs]
tokenized_docs


[['machine', 'learning', 'is', 'powerful'],
 ['deep', 'learning', 'is', 'a', 'branch', 'of', 'machine', 'learning'],
 ['this',
  'document',
  'is',
  'about',
  'learning',
  'representations',
  'of',
  'text']]

## 3. Vocabulary

The vocabulary is the set of unique tokens across all documents.

**Task:** Build `vocab`, `token2idx` and `idx2token`.


In [17]:
vocab = sorted(set(t for doc in tokenized_docs for t in doc))
token2idx = {t: i for i, t in enumerate(vocab)}
idx2token = {i: t for t, i in token2idx.items()}

print("Vocabulary size:", len(vocab))
print("First terms:", vocab[:25])


Vocabulary size: 13
First terms: ['a', 'about', 'branch', 'deep', 'document', 'is', 'learning', 'machine', 'of', 'powerful', 'representations', 'text', 'this']


## 4. Term Frequency (TF)

Term Frequency (TF) measures how important a word is **within a single document**.

The basic idea is:
- Words that appear many times in a document are likely to be important for that document.
- However, longer documents naturally contain more words, so we normalize by the document length.

We define the normalized TF as:

\[
tf(t, d) = \frac{\text{count}(t \in d)}{|d|}
\]

where:
- \(t\) is a term (word)
- \(d\) is a document
- \(|d|\) is the number of tokens in the document

This normalization allows fair comparison between documents of different lengths.


In [18]:
def tf_vector(tokens, token2idx):
    vec = np.zeros(len(token2idx), dtype=float)
    for t in tokens:
        if t in token2idx:
            vec[token2idx[t]] += 1.0
    denom = max(len(tokens), 1)
    return vec / denom

TF = np.vstack([tf_vector(doc, token2idx) for doc in tokenized_docs])
print("TF matrix shape:", TF.shape)


TF matrix shape: (3, 13)


## 5. Document Frequency (DF) and Inverse Document Frequency (IDF)

While TF captures how important a word is inside a document, it does not consider
how common the word is across the entire corpus.

### Document Frequency (DF)

Document Frequency counts in how many documents a term appears:

\[
df(t) = |\{ d \in D : t \in d \}|
\]

- Common words (e.g. *learning*) tend to have high DF.
- Rare words (e.g. *pizza*) tend to have low DF.

### Inverse Document Frequency (IDF)

Inverse Document Frequency reduces the weight of very common words and increases the
importance of rare but informative words.

We use a smoothed IDF:

\[
idf(t) = \log\left(\frac{1 + N}{1 + df(t)}\right) + 1
\]

where:
- \(N\) is the total number of documents
- Smoothing avoids division by zero and keeps values well-behaved

IDF acts as a global weighting factor shared across all documents.


In [19]:
def document_frequency(tokenized_docs, token2idx):
    df = np.zeros(len(token2idx), dtype=float)
    for doc in tokenized_docs:
        for t in set(doc):
            if t in token2idx:
                df[token2idx[t]] += 1.0
    return df

N = len(tokenized_docs)
DF = document_frequency(tokenized_docs, token2idx)
IDF = np.log((1 + N) / (1 + DF)) + 1

# quick sanity check
for term in ["learning", "pizza", "machine", "text"]:
    if term in token2idx:
        i = token2idx[term]
        print(f"{term:10s} DF={DF[i]:.0f}  IDF={IDF[i]:.3f}")


learning   DF=3  IDF=1.000
machine    DF=2  IDF=1.288
text       DF=1  IDF=1.693


## 6. TF-IDF

TF-IDF combines **local importance** (TF) with **global importance** (IDF).

\[
tfidf(t, d) = tf(t, d) \cdot idf(t)
\]

- A word gets a **high TF-IDF score** if:
  - it appears frequently in a document
  - and appears in few documents overall

- A word gets a **low TF-IDF score** if:
  - it appears in almost every document (low discriminative power)

TF-IDF is one of the most widely used techniques for classical text representation.


In [20]:
TFIDF = TF * IDF  # broadcasting multiplies each column by IDF
print("TF-IDF matrix shape:", TFIDF.shape)


TF-IDF matrix shape: (3, 13)


## 7. Document similarity with cosine similarity

Cosine similarity:
\[
\cos(\theta) = \frac{x \cdot y}{\|x\|\|y\|}
\]

**Task:** Implement cosine similarity and rank documents for a query.


In [21]:
def cosine_similarity(a, b, eps=1e-12):
    num = float(np.dot(a, b))
    den = (np.linalg.norm(a) * np.linalg.norm(b)) + eps
    return num / den

def tfidf_for_query(query: str, token2idx, idf):
    q_tokens = tokenize(query)
    q_tf = tf_vector(q_tokens, token2idx)
    return q_tf * idf

query = "machine learning"
q_vec = tfidf_for_query(query, token2idx, IDF)

sims = [cosine_similarity(q_vec, TFIDF[i]) for i in range(N)]
ranking = sorted(list(enumerate(sims)), key=lambda x: x[1], reverse=True)

print("Query:", query)
for idx, score in ranking:
    print(f"doc {idx}  sim={score:.3f}  -> {docs[idx]}")


Query: machine learning
doc 0  sim=0.638  -> Machine learning is powerful
doc 1  sim=0.546  -> Deep learning is a branch of machine learning
doc 2  sim=0.145  -> This document is about learning representations of text


### Contribution
Nombre: Álvaro Aguilar Dávila  
Asignatura: Apendizaje Automáico  
Grado: Ingeniería Informática + ADE  
Universidad: CUNEF  
Año: 2026